ENTER url below

In [1]:
url = "http://localhost:8080"

In [2]:
url += str("/query_evaluation")

In [3]:
headers = {"content-type": "application/json", "Accept-Charset": "UTF-8"}

ENTER TQL BELOW between triple qoutes

In [4]:
TQL = """$ATROPOS.SEVERE_LIVER_DISEASE.1"""

In [8]:
#TQL ='''//This is a template for a cohort study
//It has 2 cohorts (intervention and comparator)
//It has any number of outcomes that can be survival, discrete, and/or continuous in nature
//To fill out please put text into (or remove) any area with 
//If you need some examples of filling out the template please visit https://github.com/atroposhealth/consults
//Replicates study https://jmedicalcasereports.biomedcentral.com/articles/10.1186/s13256-019-2083-y

//Time Frame
var study_years = YEAR(2015, MAX)
var ages = AGE(18 YEARS, 90 YEARS)
# var eligible = INTERSECT($study_years, $ages)
# Optionally to filter by gender
var genders = ANY(GENDER="FEMALE", GENDER="MALE")
var eligible = INTERSECT($study_years, $ages, $genders)

// Define common phenotypes, if any
var diabetes = ANY($atropos.diabetes_without_chronic_complication, $atropos.diabetes_with_chronic_complication)
var diabetes_adults = FIRST MENTION(INTERSECT($diabetes, $eligible))

// intervention_treated
# any antibiotics
var abx = ATC="J01"
var fluoroquinolones = ATC="J01MA"
var washout = 3 MONTHS

var fluoroquinolones_adults = BEFORE($diabetes_adults, $fluoroquinolones*)
# No prior antibiotics (sa or fluoroquinolones) within washout period
var fluoroquinolones_newusers = FIRST MENTION(NO START OF($abx) LESS THAN $washout BEFORE START OF($fluoroquinolones_adults*))
var intervention_treated = $fluoroquinolones_newusers

// intervention_control
# systemic antibiotics excluding fluroquinolones
var sa = INTERSECT($abx, INVERT($fluoroquinolones))
var sa_adults = BEFORE($diabetes_adults, $sa*)
# No prior antibiotics (sa or fluoroquinolones) within washout period
var sa_newusers = FIRST MENTION(NO START OF($abx) LESS THAN $washout BEFORE START OF($sa_adults*))
var intervention_control = $sa_newusers


// Set combined cohort (this is not final as we may have to set additional time constraints)
######### You may not need to change this
var cohort_init = ANY($intervention_control, $intervention_treated)
// index_date is when intervention begins (denoted as 0 DAY)
var index_date_init = FIRST MENTION(START($cohort_init))

// Set pre_index time intervals (aka matching window) for matching confounders where index_date = 0 DAY
######### You may not need to change this
var pre_index_start = -90 DAYS
var pre_index_end = -1 DAY
# Set $min_hx = -$pre_index_start
var min_hx = 90 DAYS

// Set post_index time intervals for following up outcomes (aka outcome window) where index_date = 0 DAY
var post_index_start = 3 DAYS
var post_index_end = 180 DAYS
// If a mix of non-survival and survival outcomes,
//   set $post_index_end to the end of the non-survival outcome follow up window (e.g. 180 DAYS)
//   survival outcomes will ignore $post_index_end (unbounded follow up till RECORD END/survival outcome)
//   set $min_followup = $post_index_end (default for all cases except when ONLY survival outcomes)
var min_followup = $post_index_end
// If survival ONLY (without any other non-survival outcomes):
//   set $min_followup = 0 (i.e. no minimum followup as it is followed till RECORD END/survival outcome)

// The following windows will be computed from the above start/end variables
// RECORD START should be > $min_hx before $index_date_init to ensure a minimum history
// RECORD END should be > $post_index_end after $index_date_init to ensure a minimum follow up
######### You may not need to change this
var cohort_with_min_hx = START OF(RECORD START) MORE THAN $min_hx BEFORE START OF($index_date_init*)
var cohort_with_min_fu = START OF(RECORD END) MORE THAN $min_followup AFTER START OF($index_date_init*)
var cohort = INTERSECT($cohort_with_min_hx, $cohort_with_min_fu)
var index_date = START($cohort)
var post_index_window = EXTEND BY($index_date, $post_index_start, $post_index_end)
// pre_index_window is for propensity score matching
var pre_index_window = EXTEND BY($index_date, $pre_index_start, $pre_index_end)
// comorbidity_window: before and up to index_date
var comorbidity_window = EXTEND BY($index_date, MIN, 0)


// Outcomes
# Binary outcome (i.e. present/absent of a disease, procedure, medication, lab, etc)
var hypoglycemia = ANY(ICD9=251.1, ICD9=251.2, ICD9=249.8, ICD9=250.8, ICD10=E16.0, ICD10=E16.1, ICD10=E16.2)
var outcome_hypoglycemia = FIRST MENTION(START OF($hypoglycemia*) BETWEEN $post_index_start and $post_index_end AFTER START OF($index_date))
# AFTER command synonymous with SEQUENCE
#var outcome_hypoglycemia = FIRST MENTION(SEQUENCE($index_date, $hypoglycemia*)+ (START-$post_index_end, START-$post_index_start))

# Continuous outcomes (i.e. numerical)
var a1c_lab = LABS("4548-4 []")
var outcome_a1c = LABS(TIME=$post_index_window, $a1c_lab)
var vital_bmi = VITALS("Body mass index (BMI) [Ratio]", MIN, MAX)
var baseline_bmi = VITALS(TIME=$pre_index_window, $vital_bmi)
var outcome_bmi = VITALS(TIME=$post_index_window, $vital_bmi)

# Discrete outcomes (i.e. integer values, no decimals)
var telehealth_visit = VISIT TYPE = "telehealth"
var baseline_tele.visits = COUNT(START($telehealth_visit), $pre_index_window)
var outcome_tele.visits = COUNT(START($telehealth_visit), $post_index_window)
var vital_pain = VITALS("Pain severity [Score] Visual analog score", MIN, MAX)
var outcome_pain = VITALS(TIME=$post_index_window, $vital_pain)

# Survival outcome (i.e. time to outcome, e.g. death)
# unlike in binary outcome, the bounds for survival outcome is (MIN, -$post_index_start)
var outcome_death = FIRST MENTION(START OF(DEATH*) MORE THAN $post_index_start AFTER START OF($index_date))


######### Leave CSV command mostly intact except to refer to interventions, outcomes
CSV($cohort, RECORD START, RECORD END, DEMOGRAPHICS,
### Anything in  of intervention_xxx will be in report/charts/tables (no spaces; case-insensitive)
    "intervention_ref_sa" = INTERSECT($intervention_control, $cohort),
    "intervention_fq" = INTERSECT($intervention_treated, $cohort),
    "index_date" = $index_date,
### suffixes must be exactly bin/cont/surv for binary/continuous/survival respectively
### outcome_xxx_bin must not have additional _ as separators. Use . as separators if you must
    "baseline_bmi.before_cont" = $baseline_bmi,
    "baseline_tele.visits.before_disc" = $baseline_tele.visits,
    "outcome_bmi.after_cont" = $outcome_bmi,
    "outcome_pain_disc" = $outcome_pain,
    "outcome_tele.visits.after_disc" = $outcome_tele.visits,
    "outcome_hypoglycemia_bin" = $outcome_hypoglycemia,
    "outcome_a1c_cont" = $outcome_a1c,
    "outcome_death_surv" = $outcome_death,
######### Leave the following unchanged
    "comorb_malignancy" = FIRST MENTION(INTERSECT($atropos.malignancy_non_skin, $comorbidity_window)),
    "comorb_metastatic.solid.tumor" = FIRST MENTION(INTERSECT($atropos.metastatic_solid_tumour, $comorbidity_window)),
    "comorb_diabetes" = FIRST MENTION(INTERSECT($atropos.diabetes_without_chronic_complication, $comorbidity_window)),
    "comorb_diabetes.with.complications" = FIRST MENTION(INTERSECT($atropos.diabetes_with_chronic_complication, $comorbidity_window)),
    "comorb_congestive.heart.failure" = FIRST MENTION(INTERSECT($atropos.congestive_heart_failure, $comorbidity_window)),
    "comorb_myocardial.infarction" = FIRST MENTION(INTERSECT($atropos.myocardial_infarction, $comorbidity_window)),
    "comorb_peripheral.vascular.disease" = FIRST MENTION(INTERSECT($atropos.peripheral_vascular_disease, $comorbidity_window)),
    "comorb_chronic.pulmonary.disease" = FIRST MENTION(INTERSECT($atropos.chronic_pulmonary_disease, $comorbidity_window)),
    "comorb_cerebrovascular.disease" = FIRST MENTION(INTERSECT($atropos.cerebrovascular_disease, $comorbidity_window)),
    "comorb_dementia" = FIRST MENTION(INTERSECT($atropos.dementia, $comorbidity_window)),
    "comorb_renal.disease" = FIRST MENTION(INTERSECT($atropos.renal_disease, $comorbidity_window)),
    "comorb_mild.liver.disease" = FIRST MENTION(INTERSECT($atropos.mild_liver_disease, $comorbidity_window)),
    "comorb_severe.liver.disease" = FIRST MENTION(INTERSECT($atropos.severe_liver_disease, $comorbidity_window)),
    "comorb_peptic.ulcer.disease" = FIRST MENTION(INTERSECT($atropos.peptic_ulcer_disease, $comorbidity_window)),
    "comorb_rheumatic.disease" = FIRST MENTION(INTERSECT($atropos.rheumatic_disease, $comorbidity_window)),
    "comorb_hemiparaplegia" = FIRST MENTION(INTERSECT($atropos.hemiparaplegia, $comorbidity_window)),
    "comorb_hiv" = FIRST MENTION(INTERSECT($atropos.hiv, $comorbidity_window)),
    "n_encounters_pre"=COUNT(START(ENCOUNTERS), $pre_index_window),
    "icd9" = COUNT(START(ORIGINAL(ICD9)), $pre_index_window),
    "icd10" = COUNT(START(ORIGINAL(ICD10)), $pre_index_window),
    "cpt" = COUNT(START(CPT), $pre_index_window),
    "rx" = COUNT(START(RX), $pre_index_window))'''

In [5]:
%%capture
import json
import re
import requests
TQL.replace('"', '\\"')

In [6]:
%%capture
d1 ={"query":TQL}
d2 =str(d1)
print(d2)
r = requests.post(url, data=d2, headers=headers)
data = r.json()

In [7]:
data

{'originalQuery': '$ATROPOS.SEVERE_LIVER_DISEASE.1',
 'dataset': {'version': "VERSION='3.4.876'|DATASET='SHC'",
  'name': 'SHC',
  'totalCount': 100000},
 'datasetPatientCount': 100000,
 'lines': [{'text': 'UNION(ICD9="456.0", ICD9="456.1", ICD9="456.2", ICD9="572.2", ICD9="572.3", ICD9="572.4", ICD9="572.5", ICD9="572.6", ICD9="572.7", ICD9="572.8", ICD10="I85.0", ICD10="I85.9", ICD10="I86.4", ICD10="I98.2", ICD10="K70.4", ICD10="K71.1", ICD10="I72.1", ICD10="K72.9", ICD10="K76.5", ICD10="K76.6", ICD10="K76.7")',
   'count': 6,
   'children': [{'text': 'ICD9="456.0"',
     'count': 0,
     'percentageRelativeToTotalDataset': 0.0},
    {'text': 'ICD9="456.1"',
     'count': 0,
     'percentageRelativeToTotalDataset': 0.0},
    {'text': 'ICD9="456.2"',
     'count': 0,
     'percentageRelativeToTotalDataset': 0.0},
    {'text': 'ICD9="572.2"',
     'count': 1,
     'percentageRelativeToTotalDataset': 0.0001},
    {'text': 'ICD9="572.3"',
     'count': 1,
     'percentageRelativeToTotalD